In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from features import coord_to_postcode

In [ ]:
df = pd.read_csv("../data/housing.csv")
df["income_cat"] = pd.to_numeric(pd.cut(df["median_income"], bins=[0.,1.5,3.0,4.5,6.,np.inf],labels=[1,2,3,4,5]), errors='coerce').astype('Int64')
df["houseid"]=df.index
df = df.sample(10)

In [ ]:
df["zipcode"] = df.apply(lambda x : coord_to_postcode.coord2zipcode(x), axis = 1)
df.drop(["longitude", "latitude"], axis=1, inplace=True)

In [ ]:
os.environ['HOPSWORKS_PROJECT']="ondemad_feature"
os.environ['HOPSWORKS_HOST']="8c7943b0-a2d3-11ed-8cee-11db34df42ef.cloud.hopsworks.ai"
os.environ['HOPSWORKS_API_KEY']="rYKP15t4AHy0x9Xp.5EhDC1aatYdbm5JG4f8SCZzGTFXH254PVZ33BsiGjJ8KRU3vMifnGAU5idZWRLtP"

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

In [ ]:
profile_fg = fs.get_or_create_feature_group(
    name="housing_fg",
    version=1,
    description="housing data",
    primary_key = ["houseid"],
    online_enabled=True
)
profile_fg.insert(df)